In [ ]:
import requests
import pandas as pd 
import folium
from folium import FeatureGroup, features, CircleMarker
import numpy as np
from matplotlib import colors as mcolors
from haversine import haversine

In [ ]:
def popup(i,l,t,show=False):
    return folium.Popup('#%d [%f,%f]\n%s' % (i,l[0],l[1],str(t)),show=show)


In [ ]:
query=\
"""/*
This is an example Overpass query.
Try it out by pressing the Run button above!
You can find more examples with the Load tool.
*/
[out:json];
way
  [highway][tunnel][bicycle=no]
  (46.37346430137335,11.211547851562498,47.184112659842015,12.60955810546875);
/*added by auto repair*/
(._;>;);
/*end of auto repair*/
out;"""

In [ ]:
r = requests.get('http://overpass-api.de/api/interpreter?data='+query)
print(r.status_code)
ores = r.json()

In [ ]:
tunnels =[e for e in ores['elements'] if e['type']=='way']


In [ ]:
nodes = [e for e in ores['elements'] if e['type']=='node']

In [ ]:
tunnelnodes = []
for tunnel in tunnels:
    nt = []
    for nodeid in tunnel['nodes']:
        for node in nodes:
            if node['id'] == nodeid:
                nt.append([node['lat'],node['lon']])
    tunnelnodes.append(nt)

In [ ]:
len(tunnelnodes)

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
for i,tunnel in enumerate(tunnels):
    folium.PolyLine(tunnelnodes[i],color='red').add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
mypath='./2019-08-04/Tracks/'

In [ ]:
racers = pd.read_pickle('racers.pkl')

In [ ]:
route={}
for racer in racers['Name']:
    try:
        ll =[l[0] for i,l in enumerate(pd.read_pickle(mypath+racer+'.pkl').values.tolist())]
        route[racer] = ll
    except:
        print (racer)

In [ ]:
mycolors = list(mcolors.cnames.keys())

In [ ]:
ncolor =len(mycolors)

In [ ]:
1000 % ncolor

In [ ]:
def popup(name,bib,show=False):
    return folium.Popup('%s\%s' % (name,bib) ,show=show)

In [ ]:
m = folium.Map(tiles='OpenStreetMap')
folium.PolyLine(tunnelnodes[14],color='red').add_to(m)
[CircleMarker(l).add_to(m) for l in tunnelnodes[14]]
CircleMarker(tunnelnodes[14][0],color='red').add_to(m)
CircleMarker(tunnelnodes[14][-1],color='red').add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
tunnellength = [sum(list(map(lambda x,y:haversine(x,y),tunnel[1:],tunnel[:-2]))) for tunnel in tunnelnodes]

In [ ]:
bg =[]
for k,racer in racers.iterrows():
    try:
        if racer['Bib']=='CC1':
            continue
        for i,l in enumerate(route[racer['Name']]):
            for j,tunnel in enumerate(tunnelnodes):
                d = haversine(l,tunnel[0],'km')
                if d < 2:
                    bg.append([j,d,l,i,racer['Name'],racer['Bib']])
                d = haversine(l,tunnel[-1],'km')                    
                if d < 2:
                    bg.append([j,d,l,i,racer['Name'],racer['Bib']])
    except:
        print(racer['Name'])

       

In [ ]:
bg =[]
for k,racer in racers.iterrows():
    try:
        if racer['Bib']=='CC1':
            continue
        for i,l in enumerate(route[racer['Name']]):
            for j,tunnel in enumerate(tunnelnodes):
                a,b = haversine(l,tunnel[0],'km'), haversine(l,tunnel[-1],'km')
                if a < 2 or b <2: # is he close to the tunnel
                    
                    
                    bg.append([j,d,l,i,racer['Name'],racer['Bib']])
                d = haversine(l,tunnel[-1],'km')                    
                if d < 2:
                    bg.append([j,d,l,i,racer['Name'],racer['Bib']])
    except:
        print(racer['Name'])

In [ ]:
bg

In [ ]:
bgt =list(set([e[0] for e in bg]))
bgt

In [ ]:
tunnels[1]

In [ ]:
bgb =list(set([e[4] for e in bg]))

In [ ]:
bgb

In [ ]:
len(bgb)

In [ ]:
len(bgt)

In [ ]:
[e for e in bg if e[0]==1 and e[4]=='Clayton_Anderson']

In [ ]:
ll = route['Clayton_Anderson'][1131:1136]
m = folium.Map(tiles='OpenStreetMap')
[CircleMarker(l).add_to(m) for l in tunnelnodes[1]]
CircleMarker(tunnelnodes[1][0],color='purple').add_to(m)
CircleMarker(tunnelnodes[1][-1],color='yellow').add_to(m)
[CircleMarker(l,color='red').add_to(m) for l in ll]
folium.PolyLine(ll,color='red').add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
tunnels[1]

In [ ]:
[e for e in bg if e[0]==38 and e[4]=='Clayton_Anderson']

In [ ]:
ll = route['Clayton_Anderson'][1200:1210]
m = folium.Map(tiles='OpenStreetMap')
[CircleMarker(l).add_to(m) for l in tunnelnodes[38]]
CircleMarker(tunnelnodes[38][0],color='purple').add_to(m)
CircleMarker(tunnelnodes[38][-1],color='purple').add_to(m)
[CircleMarker(l,color='red').add_to(m) for l in ll]
folium.PolyLine(ll,color='red').add_to(m)
m.fit_bounds(m.get_bounds())
m